In [1]:
import pandas as pd
import scipy.stats as ss
import pingouin as pg

<b> Загружаем csv-файл

In [2]:
df = pd.read_csv('dating_data.csv')

<b> посмотрим на типы данных в файле

In [3]:
df.dtypes

user_id_1    int64
user_id_2    int64
group        int64
is_match     int64
dtype: object

<b> как видим, все данные числового типа. Посмотрим на статистику по данным

In [4]:
df.describe()

,user_id_1,user_id_2,group,is_match
count,14514.000000,14514.000000,14514.000000,14514.000000
mean,500.220132,501.843737,0.669836,0.333678
std,288.703825,287.250879,0.470288,0.471542
min,1.000000,1.000000,0.000000,0.000000
25%,253.000000,255.000000,0.000000,0.000000
50%,504.000000,500.000000,1.000000,0.000000
75%,751.000000,752.750000,1.000000,1.000000
max,1000.000000,1000.000000,1.000000,1.000000


/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


<b> user_id_1 и user_id_2 - номера юзеров, откликающихся друг на друга в соцсетях. Group и is_match - категориальные переменные. Посмотрим, как себя ведут данные для случайно взятого юзера

In [5]:
df.query('user_id_1==423')

,user_id_1,user_id_2,group,is_match
2,423,677,0,0
1075,423,271,0,1
3239,423,875,0,0
3251,423,628,0,0
4911,423,745,0,0
5347,423,64,0,0
5360,423,285,0,0
8915,423,215,0,0
9563,423,438,0,0
9895,423,150,0,0


In [6]:
df.query('group==0').agg({'user_id_1': 'nunique'})

user_id_1    501
dtype: int64

In [7]:
df.query('group==1').agg({'user_id_1': 'nunique'})

user_id_1    499
dtype: int64

<b> Случайно выбранный юзер 423 имел некие пересечения с другими юзерами, с кем-то он сходился, с кем-то нет. Отнесен к группе 1. <br> В двух выборках практически одинаковое число пользователей. <br> Разобьем датафрейм на две выборки в соответствии с их типом, извлечем только данные о совпадениях

In [8]:
df_group_0 = df.query('group==0').is_match
df_group_1 = df.query('group==1').is_match

<b> Посмотрим на характеристики выборок

In [9]:
df_group_0.describe()

count    4792.000000
mean        0.194908
std         0.396171
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: is_match, dtype: float64

In [10]:
df_group_1.describe()

count    9722.000000
mean        0.402078
std         0.490343
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: is_match, dtype: float64

<b> в выборке со старым алгоритмом более чем в два раза меньше взаимодействий между пользователями (count), а также более чем в два раза меньше положительных откликов (mean). При этом, учитывая, что величина mean является относительной, а не абсолютной величиной, увеличение числа взаимодействий при прочих равных обстоятельствах не повлияло бы на изменение значения mean. Таким образом уже заранее можно предполагать, что изменение алгоритма качественно повлияло на долю положительных мэтчей, осталось это доказать. <br> для проведения тестирования воспользуемся t-тестом. <br> Нулевая гипотеза - введение нового алгоритма статистически не повлияло на выборки

In [11]:
ss.ttest_ind(df_group_0, df_group_1)

Ttest_indResult(statistic=-25.439401464646135, pvalue=1.044367615450176e-139)

<b> Как видно, значение p-value гораздо меньше 0.05, таким образом, можно говорить о статистически значимом различии. Нулевая гипотеза отклонена, введение нового алгоритма улучшило качество поиска анкет. <br> Для проверки выполним дисперсионный анализ в scipy и pingouin

In [12]:
ss.f_oneway(df_group_0, df_group_1)

F_onewayResult(statistic=647.1631468794401, pvalue=1.0443676154453554e-139)

In [13]:
pg.anova(data=df, dv="is_match", between="group")

,Source,ddof1,ddof2,F,p-unc,np2
0,group,1,14512,647.163147,1.044368e-139,0.042691


<b> полученные значения p-value практически идентичны

In [14]:
pd.crosstab(df.group, df.is_match)

is_match,0,1
group,,
0,3858,934
1,5813,3909


<b> сравним категориальные переменные по хи-квадрат

In [15]:
from scipy.stats import chi2_contingency, chi2

In [16]:
stat, p, dof, expected = chi2_contingency(pd.crosstab(df.group, df.is_match))

In [17]:
stat, p

(618.6889141576198, 1.4418299163662586e-136)

<b> величина p-value значительно меньше 0.05, нулевая гипотеза аналогично отклонена